In [ ]:
#importing the necessary files
from nltk.tokenize import word_tokenize
import re
import glob
import os
import pickle
import nltk
from nltk.corpus import stopwords
from num2words import num2words
from math import *
import numpy as np
from nltk.stem import PorterStemmer

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
#initiliasing useful libraries for stemming and stop word removal
obj = PorterStemmer()
stop_words=set(stopwords.words('english'))

In [ ]:
#this function will convert the number to words
def numToWords(words):
    for i,w in enumerate(words):
        if w.isdigit() and len(w)<4:
            words[i]=num2words(int(w))
    return words

In [ ]:
overall_files={}#this will store the all the preprocessed words at every document index
file_index={} #this will store the filename and length of the document in words at every document index
master_unique_words=set() #this set will contain all the unique words in whole of the corpora

In [ ]:
#here we will preprocess the document by performing certain operations such as tokenization, stemming , making all the
#charaters smaller case , removing non-ascii characters, making numbers to words and stpring all the relevent things
#in the above mentioned dictionaries and set
def preprocess(file_name,idx):
   
    file=open("english-corpora/"+file_name,'r',encoding='utf-8')#reading the file
    text=file.read()
    text=text.replace("\n"," ")#removing the newline character
    text=text.replace("\t"," ") #removing the tab character
    encoded = text.encode("ascii", "ignore")#removing the ascii characters
    decoded = encoded.decode()   
    text_lower=str(np.char.lower(decoded))#making characters lower case    
    text_new=text_lower.replace("'","")#removing the apostrophe 
    text_new = re.sub(r'[^\w\s]',' ',text_new)
    token_list=word_tokenize(text_new) #tokenization
    word_list=numToWords(token_list)
    word_list=' '.join(word_list)
    word_list=re.sub(r'[^\w\s]',' ',word_list)
    word_list=word_tokenize(word_list)
    word_list=[x for x in word_list if x not in stop_words]#stopwords removal
    words_list_new=list(filter(lambda x:len(x)>1,word_list))    
    stem_words = [obj.stem(w) for w in words_list_new]#stemming 
    master_unique_words.update(stem_words)
    
    overall_files[idx]=stem_words
    file_index[idx]=[file_name,len(stem_words)]

In [ ]:
idx=0
folder="english-corpora/*"#the english corpora folder should be in the same directory as this notebook
for file in glob.glob(folder):
    print(file)
    file_name=os.path.basename(file)
    preprocess(file_name,idx)
    idx+=1


In [ ]:
#print(len(master_unique_words))

In [ ]:
#this dictionary stores for every word which document it is present in and the frequency in that document
posting_lists={}
for word in master_unique_words:
    posting_lists[word]={}#here we intialising with empty structure so that we dont get keyerror

In [ ]:
#this function will return the count of word here words passed is a list and will return a dictionary
def find_words_freq_doc(words):
    freq={}
    #here we are making a set so that we dont count a word more than once
    unique_words=set(words)
    
    #here we are counting the frequency for each word in the passed list
    for word in unique_words:
        cnt=0
        for i in words:
            if i == word:
                cnt+=1
        freq[word]=cnt
    return freq

In [ ]:
#here we are iterating the overall_files and finding their frequency and storing them in the posting list
for docid in overall_files.keys():
    words=overall_files[docid]
    
    word_freq=find_words_freq_doc(words)
    
    for word in word_freq.keys():
        posting_lists[word][docid]=word_freq[word]#here we are storing the word count for every word present in the document


In [ ]:
#this dictionary will store the norms which will be used later
norm_docs={}
total_doc_size=len(file_index)
for file in overall_files.keys():
    words_list=overall_files[file]
    
    norm=0
    for word in set(words_list):
        tf=0 #for calculating term freuency
        for i in word_list:
            if i==word:
                tf+=1
        doc_freq=len(posting_lists[word])#total document frency in the document
        
        idf=log2((total_doc_size+1)/(doc_freq))#calculating IDF
        
        tf_idf=tf*idf
        
        norm+=tf_idf**2
    
    norm_value=sqrt(norm)#calculating norm
    norm_docs[file]=norm_value
    print(file)


In [ ]:
with open("posting_lists","wb") as f:
    pickle.dump(posting_lists,f)

with open("file_index","wb") as f:
    pickle.dump(file_index,f)

with open("file_word_list","wb") as f:
    pickle.dump(overall_files,f)
    
with open("file_norms","wb") as f:
    pickle.dump(norm_docs,f)
    
